# Measuring the Benefit of Boyer-Moore's Algorithm
Boyer-Moore algorithms are significantly more efficient compared to naive exact matching; here, the improvement in workload of a Boyer-Moore algorithm was quantified by measuring the number of character comparisons performed and the number of alignments tested.

This exercise was part of Coursera's Algorithms for DNA Sequencing course as part of the Genomic Data Science specialization offered by the Johns Hopkins Unversity.

In [1]:
# This module provides the BoyerMoore class, which contains the preprocessing info used by the boyer moore function
from bm_preproc import *


In [2]:
# Writing the Boyer Moore function to count the number of character comparisons
def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        num_alignments += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, num_alignments, num_character_comparisons

In [3]:
# Testing out the Boyer-Moore algorithm with counts
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)
# ([40], 12, 15)

[40] 12 15


In [4]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


In [5]:
# Contrasting the Boyer-Moore algorith to a naive exact matching algorithm
# writing the naive algorithm to include a count of the number of character comparisons'

def naive_with_counts(p, t):
    occurences = []
    num_alignments = 0
    num_character_comparisons = 0
    
    for i in range(len(t) - len(p) + 1):
        num_alignments += 1
        match = True
        for j in range(len(p)):
            num_character_comparisons += 1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurences.append(i)
    return occurences, num_alignments, num_character_comparisons
        

In [6]:
# Testing out the cNaive algorithm with charact comparison counts
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


In [7]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)


[0, 19] 20 35


Indeed, results suggested that the Boyer Moore algorithm is more comoputationally efficient than naive exact matching, as it required fewer character comparisons and alignments in both examples!!

In [8]:
# For a more realistic example, testing out the alignment algorithm on a reference genome
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [9]:
genome = readGenome('chr1.GRCh38.excerpt.fasta')
genome[:200]

'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAAGAAAGCAATTTTTGCTGCTAACCTAACATTTCACAATGTCTGGAGACATTTACAGTTCCCACAACCTATGGCAGTTACTGGCATCTACTAGAGGTCAGAG'

In [10]:
# 1 Naive exact matching with a sample pattern sequence
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = genome
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print('Naive Exact Matching \n')
print('Number of occurences: ', occurrences)
print('Number of alignments: ', num_alignments)
print('Number of character comparisons: ', num_character_comparisons)

Naive Exact Matching 

Number of occurences:  [56922]
Number of alignments:  799954
Number of character comparisons:  984143


In [11]:
# 2 Naive exact matching with a sample pattern sequence

p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = genome
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print('Naive Exact Matching \n')
print('Number of occurences: ', occurrences)
print('Number of alignments: ', num_alignments)
print('Number of character comparisons: ', num_character_comparisons)

Naive Exact Matching 

Number of occurences:  [56922]
Number of alignments:  799954
Number of character comparisons:  984143


In [12]:
# 3 Boyer Moore with a sample pattern sequence

p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = genome
p_bm = BoyerMoore(p)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print('Boyer Moore \n')
print('Number of occurences: ', occurrences)
print('Number of alignments: ', num_alignments)
print('Number of character comparisons: ', num_character_comparisons)


Boyer Moore 

Number of occurences:  [56922]
Number of alignments:  127974
Number of character comparisons:  165191


In [25]:
# Boyer Moore improvements:
print('Boyer Moore Improvements')
print(100 - round(127974/799954*100), '% fewer alignments')
print(100 - round(165191/984143*100), '% fewer character comparisons')

Boyer Moore Improvements
84 % fewer alignments
83 % fewer character comparisons


Boyer Moore found the same number of occurrences of the pattern string in the reference text, while using significantly fewer alignments and comparisons!

In [14]:
from kmer_index import *

In [15]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        num_alignments += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

In [16]:
# Approximate matching using boyer moore algorithm
def approximate_match(p, t, n):
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    for i in range(n+1): # for each partition
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        matches = boyer_moore(p[start:end], p_bm, t)
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            
            mismatches = 0
            for j in range(0,start):
                text_start = m - start
                if not p[j] == t[text_start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                text_start = m - start
                if not p[j] == t[text_start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            
            if mismatches <= n:
                all_matches.add(m-start)
    return list(all_matches)

In [17]:
# testing out approximate matching
p = 'AACTTG'
t = 'CACTTAATTTG'
print(approximate_match(p, t, 2))

[0, 5]


In [18]:
def queryIndex(p, t, index):
    ''' 
    Input pattern to query, text to search and index of kmers to use
    outputs a list of offset matches
    '''
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:
            offsets.append(i)
    return offsets

In [19]:
def naive_2mm(p, t):
    occurences = []
    for i in range(len(t) - len(p) + 1):
        num_mismatches = 0
        match = True
        for j in range(len(p)):
            if not t[i+j] == p[j]:
                num_mismatches += 1
                if num_mismatches >2:
                    match = False
                    break
        if match:
            occurences.append(i)
    return occurences

In [20]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = genome
print(naive_2mm(p,t))
print(len(set(naive_2mm(p, t))))

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]
19


In [21]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = genome
index = Index(t, k=8)
index_hits_to_check = index.query_24(p)
final_hits = []
for hit_i in index_hits_to_check:
    mismatches = 0
    for i in range(len(p)):
        if t[hit_i+i] != p[i]:
            mismatches += 1
        if mismatches > 2:
            break
    if mismatches < 3:
        final_hits.append(hit_i)
final_hits = list(set(final_hits))
print(final_hits)
print(len(final_hits))

[273669, 681737, 717706, 262042, 635931, 84641, 160162, 724927, 657496, 160729, 56922, 191452, 551134, 747359, 421221, 147558, 364263, 465647, 429299]
19


In [22]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = genome
index = Index(t, k=8)
index_hits_to_check = index.query_24(p)
# final_hits = []
# for hit_i in index_hits_to_check:
#     mismatches = 0
#     for i in range(len(p)):
#         if t[hit_i+i] != p[i]:
#             mismatches += 1
#         if mismatches > 2:
#             break
#     if mismatches < 3:
#         final_hits.append(hit_i)
# final_hits = list(set(final_hits))
print(index_hits_to_check)
print(len(index_hits_to_check))

[56922, 57056, 83720, 84641, 147558, 160729, 191452, 262042, 364263, 657496, 681737, 717706, 725061, 18870, 56922, 160162, 262042, 273669, 282004, 364263, 421221, 429299, 465647, 472634, 489438, 551134, 621362, 657496, 717706, 724927, 18733, 19166, 22397, 22532, 23003, 23138, 43127, 56922, 67363, 83720, 83863, 84641, 84775, 108110, 128994, 147558, 160729, 175310, 185996, 187655, 191452, 205381, 251090, 251224, 262042, 273669, 282004, 322735, 364263, 364396, 421221, 429299, 454332, 465647, 471966, 480501, 480642, 523085, 551134, 551827, 572196, 588478, 595541, 613459, 621491, 632305, 635931, 646488, 651523, 657496, 674056, 681737, 707151, 717706, 719418, 724927, 746620, 747359, 747495, 760489]
90


In [23]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t = genome
index = SubseqIndex(t, k=8, ival=3)
index_hits_to_check = index.query(p)
print(index_hits_to_check)
print(len(index_hits_to_check))


[56922, 67486, 83863, 84641, 84775, 124024, 147558, 191452, 199607, 262042, 262174, 273669, 322735, 364263, 421354, 454332, 465647, 471966, 472634, 489019, 558456, 579737, 596898, 635931, 651523, 657496, 658702, 681737, 707151, 712449, 717706, 719418, 719557, 746620, 747359]
35


In [24]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
index = SubseqIndex(t, 8, 3)
index_hits_to_check = index.query(p)
final_hits = []
for hit_i in index_hits_to_check:
    mismatches = 0
    for i in range(len(p)):
        if t[hit_i+i] != p[i]:
            mismatches += 1
        if mismatches > 2:
            break
    if mismatches < 3:
        final_hits.append(hit_i)
final_hits = list(set(final_hits))
print(final_hits)
print(len(final_hits))

[84641, 273669, 147558, 364263, 681737, 717706, 465647, 747359, 657496, 56922, 635931, 191452, 262042]
13
